# Construct the `experiment_info` Dictionary

This notebook is meant for developers that would like to contribute to the utilities of the repository. Furthermore, it provides means for the maintainers of the repo to easily update the dictionary when new data comes in. 

It details how the README files are processed and a dictionary is created that contains the information of the different experiments. The dictionary is written to a text file automatically.

In [ ]:
import os
import sys
import importlib

import pandas as pd


# Absolute path to MaCFP matl repository:
# "D:\Git\MaCFP_matl_MyFork\matl-db"
mafp_matl_root = os.path.join("d:/", "Git", "MaCFP_matl_MyFork", "matl-db")
# mafp_matl_root = os.path.join("d:/", "Git", "MaCFP_matl", "matl-db")


# Add path to where Python looks for modules to import..
base_func_script = os.path.join(mafp_matl_root, "Utilities")
sys.path.insert(1, base_func_script)

# Import base_functions script.
import basic_functions as base_f
# Re-import base_functions script.
importlib.reload(base_f)

In [ ]:
# Import general information needed for this notebook.

# Path to the PMMA data.
pmma_path = os.path.join(mafp_matl_root, "Non-charring", "PMMA")
umet_path = os.path.join(pmma_path, "UMET")

## Get All README Files Containing TGA Data

First all institute labels are taken from the directory labels.

In [ ]:
# Get all institute labels from dictionary names.
institutes = list()
institutes = [ f.name for f in os.scandir(pmma_path) if f.is_dir() ]

# Check results, remove ";".
institutes;

Then all README files are read from the institute directories. Each file is checked it it contains information on TGA experiments. 

In [ ]:
# Get all README files that contain information on the TGA experiments.
tga_readme_files = base_f.get_exp_readme_files(institutes=institutes,
                                               base_path=pmma_path, 
                                               experiment_key="TGA")

# Check results, remove ";".
tga_readme_files.keys();

## Extract the Information on the TGA Experiments
Now, the README files are parsed and the information on the TGA experiments is extracted.

In [ ]:
experiment_info = {"TGA": dict()}
# experiment_info = dict()

# for institute in tga_readme_files.keys():
for institute in ["UMET"]:
    # Get README file content.
    readme_lines = tga_readme_files[institute]
    
    # Extract institute label and name.
    institute_info = base_f.get_institute(readme_lines)
    print(institute_info[0])
    
    # Extract the TGA experiment information.
    experiment_lines = base_f.read_experiment_lines(readme_lines, 
                                                    start_marker_a="TGA", 
                                                    start_marker_b="###", 
                                                    end_marker="###")
    
    # Extract test conditions summary table.
    test_cond_df = base_f.read_test_condition_table(experiment_lines)
    
    institute_test_info = base_f.build_tga_dict(experiment_lines,
                                                institute_name_info=institute_info,
                                                exp_table_df=test_cond_df,
                                                tga_base_dict=base_f.experiment_template["TGA_base"],
                                                material_path=pmma_path)
    
    experiment_info["TGA"] = institute_test_info

In [ ]:
experiment_info;

## Save Experiment Information Dictionary
Now, the extracted information of the TGA experiments, that was stored in the dictionary, ist saved as a text file (Python). Thus, for further work, the dictionary can be imported from the Python file and the information in it is readily accessible. 

In [ ]:
# A text file "ExperimentInformation.py" is created and the content of
# the dictionary is written to it.
with open('ExperimentInformation.py','w', encoding='utf8') as exp_info_file:
    exp_info_file.write("matl_db_info = " + str(experiment_info))

In [ ]:
# Demonstration as to how to read the dictionary again.

# Import of the Python file.
import ExperimentInformation as exp_info
# Re-import base_functions script 
# (neccessary for changes to take effect without kernel restart).
importlib.reload(exp_info)

# Get path of a CSV file containing data series of a TGA experiment.
file_name = exp_info.matl_db_info["TGA"]["UMET"]["UMET_TGA_N2_1"]["path"]

# Read file as a Pandas DataFrame and show the first five lines.
pd.read_csv(os.path.join(mafp_matl_root, file_name)).head()

## Build README from Dictionary
Here, an example is provided that README files can also be built from a dictionary.

*Needs further development...*